In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import os 
from time import time
from time import sleep

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'10',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '32b87023-5bb8-42e8-a360-972adc1ca1f7',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)


In [ ]:
type(data)

In [ ]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:

df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df


In [ ]:

def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'10',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '32b87023-5bb8-42e8-a360-972adc1ca1f7',
    }
    
    session = Session()
    session.headers.update(headers)
    
    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)

    df2 = pd.json_normalize(data['data'])
    df2['timestamp'] = pd.to_datetime('now')
    df = pd.concat([df, df2], ignore_index=False)

    # Use this if you want to create a csv and append data to it
    # df = pd.json_normalize(data['data'])
    # df['timestamp'] = pd.to_datetime('now')
    # df

    # if not os.path.isfile(r'C:\Users\HP\OneDrive\Documents\Documents - Copy\Data Analysis Projects\Python Training\API Training\API.csv'):
    #     df.to_csv(r'C:\Users\HP\OneDrive\Documents\Documents - Copy\Data Analysis Projects\Python Training\API Training\API.csv', header='column_names')
    # else:
    #     df.to_csv(r'C:\Users\HP\OneDrive\Documents\Documents - Copy\Data Analysis Projects\Python Training\API Training\API.csv', mode='a', header=False)
        
    #Then to read in the file: df = pd.read_csv(r'C:\Users\alexf\OneDrive\Documents\Python Scripts\API.csv')


In [ ]:

for i in range(250):
    api_runner()
    print("API Runner Completed")
    sleep(60)
exit()


In [ ]:
df

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
df

In [ ]:

df3 = df.groupby('name', sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()
df3


In [ ]:

df4 = df3.stack()
df4


In [ ]:
type(df4)

In [ ]:
df5 = df4.to_frame(name='values')
df5

In [ ]:
df5.count()

In [ ]:

index = pd.Index(range(60))

#df6 = df5.set_index(index)
df6 = df5.reset_index()
df6


In [ ]:

df7 = df6.rename(columns={'level_1': 'percent_change'})
df7


In [ ]:

df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d'],['1h','24h','7d','30d','60d','90d'])
df7


In [ ]:

plot = sns.catplot(x='percent_change', y='values', hue='name', data=df7, kind='point')
plot.fig.set_size_inches(12, 6)  # width, height in inches
plt.xlabel('Percent Change', fontsize=12)
plt.ylabel('Values', fontsize=12)
plt.show()


In [ ]:

barchat = sns.catplot(x='percent_change', y='values', hue='name', data=df7, kind='bar')
barchat.fig.set_size_inches(12, 6)

plt.xlabel('Percent Change', fontsize=12)
plt.ylabel('Values', fontsize=12)
plt.show()


In [ ]:

plt.figure(figsize=(12,6))
sns.barplot(x='percent_change', y='values', data=df7)
plt.xlabel('Percent Change', fontsize=12)
plt.ylabel('Values', fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:

plt.figure(figsize=(12,6))
sns.barplot(y='percent_change', x='values', data=df7)
plt.ylabel('Percent Change', fontsize=12)
plt.xlabel('Values', fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:

df_grouped = df7.groupby(['percent_change', 'name'])['values'].sum().unstack()
df_grouped.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.xlabel('Percent Change')  # Optional
plt.ylabel('Values')          # Optional
plt.tight_layout()
plt.show()


In [ ]:

plt.figure(figsize=(12,6))
sns.countplot(x='percent_change', data=df7)
plt.xlabel('Percent Change', fontsize=12)
plt.tight_layout()
plt.show()
